In [33]:
import pandas as pd
import numpy as np

In [34]:
#reading in all main five datasets
Educators= pd.read_csv("Prepared_Educators.csv")
EPI= pd.read_csv("Prepared_EPI.csv")
Expenditures= pd.read_csv("Prepared_Expenditures.csv")
GraduationRate = pd.read_csv("Prepared_GraduationRate.csv")
ParticipationRate = pd.read_csv("Prepared_ParticipationRate.csv")

In [35]:
#investigating the size of each dataset to get the right order of left joins
print(Educators.shape)
print(EPI.shape)
print(Expenditures.shape)
print(GraduationRate.shape)
print(ParticipationRate.shape)

(231, 5)
(363, 10)
(616, 5)
(60, 4)
(231, 5)


In [36]:
df_merged1 = Educators.merge(ParticipationRate, on=['REF_DATE', 'GEO'], how='left')

In [37]:
df_merged1.shape

(231, 8)

In [38]:
df_merged1

,REF_DATE,GEO,Full-time educators,Part-time educators,"Total, work status",College,Elementary and/or High School,University
0,2002,Alberta,26970.0,7788.0,34761.0,NaN,NaN,NaN
1,2002,British Columbia,28572.0,8577.0,37149.0,NaN,NaN,NaN
2,2002,Canada,283023.0,67599.0,350619.0,NaN,NaN,NaN
3,2002,Manitoba,12042.0,2097.0,14139.0,NaN,NaN,NaN
4,2002,New Brunswick,7050.0,366.0,7416.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
226,2022,Nova Scotia,11247.0,NaN,11250.0,8.0,3.0,31.0
227,2022,Ontario,134481.0,21006.0,155487.0,13.0,5.0,33.0
228,2022,Prince Edward Island,1689.0,99.0,1788.0,10.0,5.0,32.0
229,2022,Quebec,80757.0,39882.0,120639.0,25.0,4.0,26.0


In [40]:
df_merged2 = df_merged1.merge(EPI, on=['REF_DATE', 'GEO'], how='left')

In [41]:
df_merged2.shape

(231, 16)

In [42]:
df_merged2

,REF_DATE,GEO,Full-time educators,Part-time educators,"Total, work status",College,Elementary and/or High School,University,Education price index (EPI),Fees and contractual services sub-index,Instructional supplies sub-index,Non-salary sub-index,Non-teaching salaries sub-index,Salaries and wages sub-index,"School facilities, supplies and services sub-index",Teachers' salaries sub-index
0,2002,Alberta,26970.0,7788.0,34761.0,NaN,NaN,NaN,158.56,200.92,206.02,183.47,142.42,153.69,147.17,154.72
1,2002,British Columbia,28572.0,8577.0,37149.0,NaN,NaN,NaN,154.95,183.59,206.02,169.60,142.36,152.87,135.57,154.79
2,2002,Canada,283023.0,67599.0,350619.0,NaN,NaN,NaN,156.92,201.20,206.02,186.80,147.40,150.72,147.35,151.10
3,2002,Manitoba,12042.0,2097.0,14139.0,NaN,NaN,NaN,152.79,204.60,206.02,179.27,152.00,147.84,140.40,147.17
4,2002,New Brunswick,7050.0,366.0,7416.0,NaN,NaN,NaN,152.02,206.08,206.02,172.43,152.67,147.64,133.12,146.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2022,Nova Scotia,11247.0,NaN,11250.0,8.0,3.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,2022,Ontario,134481.0,21006.0,155487.0,13.0,5.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,2022,Prince Edward Island,1689.0,99.0,1788.0,10.0,5.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,2022,Quebec,80757.0,39882.0,120639.0,25.0,4.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df_merged3 = df_merged2.merge(GraduationRate, on=['REF_DATE', 'GEO'], how='left')

In [44]:
df_merged3

,REF_DATE,GEO,Full-time educators,Part-time educators,"Total, work status",College,Elementary and/or High School,University,Education price index (EPI),Fees and contractual services sub-index,Instructional supplies sub-index,Non-salary sub-index,Non-teaching salaries sub-index,Salaries and wages sub-index,"School facilities, supplies and services sub-index",Teachers' salaries sub-index,Extended-time,On-time
0,2002,Alberta,26970.0,7788.0,34761.0,NaN,NaN,NaN,158.56,200.92,206.02,183.47,142.42,153.69,147.17,154.72,NaN,NaN
1,2002,British Columbia,28572.0,8577.0,37149.0,NaN,NaN,NaN,154.95,183.59,206.02,169.60,142.36,152.87,135.57,154.79,NaN,NaN
2,2002,Canada,283023.0,67599.0,350619.0,NaN,NaN,NaN,156.92,201.20,206.02,186.80,147.40,150.72,147.35,151.10,NaN,NaN
3,2002,Manitoba,12042.0,2097.0,14139.0,NaN,NaN,NaN,152.79,204.60,206.02,179.27,152.00,147.84,140.40,147.17,NaN,NaN
4,2002,New Brunswick,7050.0,366.0,7416.0,NaN,NaN,NaN,152.02,206.08,206.02,172.43,152.67,147.64,133.12,146.78,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2022,Nova Scotia,11247.0,NaN,11250.0,8.0,3.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,2022,Ontario,134481.0,21006.0,155487.0,13.0,5.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,2022,Prince Edward Island,1689.0,99.0,1788.0,10.0,5.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,2022,Quebec,80757.0,39882.0,120639.0,25.0,4.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
merged_final = df_merged3.merge(Expenditures, on=['REF_DATE', 'GEO'], how='left')

In [46]:
merged_final

,REF_DATE,GEO,Full-time educators,Part-time educators,"Total, work status",College,Elementary and/or High School,University,Education price index (EPI),Fees and contractual services sub-index,...,Non-salary sub-index,Non-teaching salaries sub-index,Salaries and wages sub-index,"School facilities, supplies and services sub-index",Teachers' salaries sub-index,Extended-time,On-time,Teachers salaries,Total expenditures,Total operating expenditures
0,2002,Alberta,26970.0,7788.0,34761.0,NaN,NaN,NaN,158.56,200.92,...,183.47,142.42,153.69,147.17,154.72,NaN,NaN,124919.00,247367.00,229994.00
1,2002,British Columbia,28572.0,8577.0,37149.0,NaN,NaN,NaN,154.95,183.59,...,169.60,142.36,152.87,135.57,154.79,NaN,NaN,245399.00,476518.00,427825.00
2,2002,Canada,283023.0,67599.0,350619.0,NaN,NaN,NaN,156.92,201.20,...,186.80,147.40,150.72,147.35,151.10,NaN,NaN,1297585.00,2748251.00,2530258.00
3,2002,Manitoba,12042.0,2097.0,14139.0,NaN,NaN,NaN,152.79,204.60,...,179.27,152.00,147.84,140.40,147.17,NaN,NaN,53686.00,100212.00,95034.00
4,2002,New Brunswick,7050.0,366.0,7416.0,NaN,NaN,NaN,152.02,206.08,...,172.43,152.67,147.64,133.12,146.78,NaN,NaN,153282.75,330971.25,302283.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2022,Nova Scotia,11247.0,NaN,11250.0,8.0,3.0,31.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,2022,Ontario,134481.0,21006.0,155487.0,13.0,5.0,33.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,2022,Prince Edward Island,1689.0,99.0,1788.0,10.0,5.0,32.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,2022,Quebec,80757.0,39882.0,120639.0,25.0,4.0,26.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
merged_final = merged_final.interpolate(method='linear', axis=0).ffill().bfill()

C:\Users\mihir\AppData\Local\Temp\ipykernel_18608\808605854.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  merged_final = merged_final.interpolate(method='linear', axis=0).ffill().bfill()


In [48]:
merged_final

,REF_DATE,GEO,Full-time educators,Part-time educators,"Total, work status",College,Elementary and/or High School,University,Education price index (EPI),Fees and contractual services sub-index,...,Non-salary sub-index,Non-teaching salaries sub-index,Salaries and wages sub-index,"School facilities, supplies and services sub-index",Teachers' salaries sub-index,Extended-time,On-time,Teachers salaries,Total expenditures,Total operating expenditures
0,2002,Alberta,26970.0,7788.0,34761.0,13.0,4.0,17.0,158.56,200.92,...,183.47,142.42,153.69,147.17,154.72,84.0,78.0,124919.00,247367.00,229994.00
1,2002,British Columbia,28572.0,8577.0,37149.0,13.0,4.0,17.0,154.95,183.59,...,169.60,142.36,152.87,135.57,154.79,84.0,78.0,245399.00,476518.00,427825.00
2,2002,Canada,283023.0,67599.0,350619.0,13.0,4.0,17.0,156.92,201.20,...,186.80,147.40,150.72,147.35,151.10,84.0,78.0,1297585.00,2748251.00,2530258.00
3,2002,Manitoba,12042.0,2097.0,14139.0,13.0,4.0,17.0,152.79,204.60,...,179.27,152.00,147.84,140.40,147.17,84.0,78.0,53686.00,100212.00,95034.00
4,2002,New Brunswick,7050.0,366.0,7416.0,13.0,4.0,17.0,152.02,206.08,...,172.43,152.67,147.64,133.12,146.78,84.0,78.0,153282.75,330971.25,302283.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2022,Nova Scotia,11247.0,10642.5,11250.0,8.0,3.0,31.0,158.08,208.37,...,191.40,151.53,149.70,151.88,149.35,91.0,83.0,11148.00,36605.00,35791.00
227,2022,Ontario,134481.0,21006.0,155487.0,13.0,5.0,33.0,158.08,208.37,...,191.40,151.53,149.70,151.88,149.35,91.0,83.0,11148.00,36605.00,35791.00
228,2022,Prince Edward Island,1689.0,99.0,1788.0,10.0,5.0,32.0,158.08,208.37,...,191.40,151.53,149.70,151.88,149.35,91.0,83.0,11148.00,36605.00,35791.00
229,2022,Quebec,80757.0,39882.0,120639.0,25.0,4.0,26.0,158.08,208.37,...,191.40,151.53,149.70,151.88,149.35,91.0,83.0,11148.00,36605.00,35791.00


In [49]:
merged_final.to_csv("Final_Education_Dataset.csv")

In [10]:
df_merged1 = Expenditures.merge(EPI, on=['REF_DATE', 'GEO'], how='left')

In [11]:
df_merged1.shape

(616, 13)

In [12]:
df_merged2 = df_merged1.merge(ParticipationRate, on=['REF_DATE', 'GEO'], how='left')

In [13]:
df_merged2.shape

(616, 16)

In [14]:
df_merged3 = df_merged2.merge(Educators, on=['REF_DATE', 'GEO'], how='left')

In [16]:
df_merged3.shape

(616, 19)

In [19]:
merged_final = df_merged3.merge(GraduationRate, on=['REF_DATE', 'GEO'], how='left')

In [20]:
merged_final.shape

(616, 21)

In [21]:
merged_final

,REF_DATE,GEO,Teachers salaries,Total expenditures,Total operating expenditures,Education price index (EPI),Fees and contractual services sub-index,Instructional supplies sub-index,Non-salary sub-index,Non-teaching salaries sub-index,...,"School facilities, supplies and services sub-index",Teachers' salaries sub-index,College,Elementary and/or High School,University,Full-time educators,Part-time educators,"Total, work status",Extended-time,On-time
0,1947,Alberta,186.00,528.00,528.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947,British Columbia,224.00,1265.00,1265.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1947,Canada,3742.00,13951.00,13951.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1947,Manitoba,106.00,556.00,556.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1947,New Brunswick,387.25,1544.25,1544.25,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,2002,Nova Scotia,352476.25,792489.75,716783.25,144.99,202.86,206.02,170.93,152.67,...,124.87,140.56,NaN,NaN,NaN,9594.0,NaN,9594.0,NaN,NaN
612,2002,Ontario,452073.00,1023249.00,924033.00,158.10,202.57,206.02,190.36,146.57,...,156.83,151.55,NaN,NaN,NaN,115914.0,14913.0,130821.0,NaN,NaN
613,2002,Prince Edward Island,421143.00,922577.50,852075.00,152.54,199.71,206.02,163.65,162.01,...,130.84,149.63,NaN,NaN,NaN,1290.0,309.0,1599.0,NaN,NaN
614,2002,Quebec,390213.00,821906.00,780117.00,159.53,200.65,206.02,191.08,150.91,...,145.16,151.95,NaN,NaN,NaN,63627.0,29457.0,93081.0,NaN,NaN
